In [18]:
import tensorflow as tf
import numpy as np
#np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow import estimator

import pandas as pd
tf.reset_default_graph()

In [19]:
def normalize(df):    
    return (df - df.min()) / (df.max() - df.min())


def denormalize(df,norm_data):    
    return (norm_data * (df.max() - df.min())) + df.min()


def preProcessForTrain():
    df = pd.read_csv('./GlobalData/globaldata.csv')
    df= df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)
    df['out']=df['out']*1
    df = df.dropna(inplace=False)
    msk = np.random.rand(len(df)) < 0.7
    df_train = df[msk]    
    df_test = df[~msk]
    X_train = normalize(df_train.drop(['out'],axis=1)).values
    y_train = df_train['out'].values
    X_test = normalize(df_test.drop(['out'],axis=1)).values
    y_test = df_test['out'].values
    return X_train, y_train, X_test, y_test


def preProcessForTest():
    df = pd.read_csv('./GlobalData/globaldata.csv')
    df= df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)
    df['out']=df['out']*1
    df = df.dropna(inplace=False)
    msk = np.random.rand(len(df)) < 1
    df_test = df[msk]
    X_test = normalize(df_test.drop(['out'],axis=1)).values
    y_test = df_test['out'].values
    return X_test, y_test


def DNNmodel():
    feat_cols=[tf.feature_column.numeric_column('x', shape=[960])]
    return tf.estimator.DNNClassifier(feature_columns=feat_cols, 
                                      n_classes=2,
                                      hidden_units=[1000, 1000, 1000,  1000, 500, 300],
                                      activation_fn=tf.nn.relu,
                                      optimizer=tf.train.GradientDescentOptimizer(0.0001),
                                      model_dir='./model1')

def training(X_train, y_train):
    feat_cols=[tf.feature_column.numeric_column('x', shape=[960])]
    deep_model=DNNmodel()
    input_fn = estimator.inputs.numpy_input_fn(x={'x':X_train}, 
                                           y=y_train,
                                           shuffle= True,
                                           num_epochs=5000,
                                           batch_size=100)
    return deep_model.train(input_fn=input_fn, steps=25000)


def evaluate(X_test, y_test):   
    input_fn_eval = estimator.inputs.numpy_input_fn( x = {'x':X_test},
                                                y =  y_test,
                                                shuffle = False)
    deep_model=DNNmodel()
    
    return dict(deep_model.evaluate(input_fn=input_fn_eval))


def predict(X_test):
    
    input_fn_eval = estimator.inputs.numpy_input_fn( x = {'x':X_test},
                                                shuffle = False)
    deep_model = DNNmodel()
    preds=list(deep_model.predict(input_fn=input_fn_eval))
    predictions = [p['class_ids'][0] for p in preds]
    pred = np.asarray(predictions)
    return pred    


def checkAccuracy(pred, y_test):
    f = pred == y_test
    T = len(f[f == True])
    F = len(f[f == False])
    error = F/(T+F) * 100
    return T, F, error 

In [20]:
X_train, y_train, X_test, y_test = preProcessForTrain()

In [21]:
train = training(X_train, y_train)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model1', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f67f3771748>, '_session_config': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_is_chief': True, '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_train_distribute': None, '_task_id': 0, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_service': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./model1/model.ckpt.
INFO:tensorflow:step = 1, loss = 69.20577
IN

In [24]:
pred = predict(X_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model1', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f687b7f44a8>, '_session_config': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_is_chief': True, '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_train_distribute': None, '_task_id': 0, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_service': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model1/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [29]:
pred = evaluate(X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model1', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6877f45cc0>, '_session_config': None, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_is_chief': True, '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_train_distribute': None, '_task_id': 0, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_service': None, '_master': ''}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-09-11:17:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model1/model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished eva

In [26]:
T


4686

In [27]:
F

3899

In [28]:
error

45.41642399534071